In [108]:
import pandas as pd
import re
import numpy as np


In [59]:
filepath = r"Resources/winemag-data-130k-v2.json"

# Loading the dataset
df = pd.read_json(filepath)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [60]:
df.count()

country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

In [61]:
# Remove the row that dont have values in the point or price columns
df2 = df.dropna(subset=['points', 'price'])
df2.count()

country                  120916
description              120975
designation               86196
points                   120975
price                    120975
province                 120916
region_1                 101400
region_2                  50292
taster_name               96479
taster_twitter_handle     91559
title                    120975
variety                  120974
winery                   120975
dtype: int64

In [70]:
df2.describe()

,points,price
count,120975.000000,120975.000000
mean,88.421881,35.363389
std,3.044508,41.022218
min,80.000000,4.000000
25%,86.000000,17.000000
50%,88.000000,25.000000
75%,91.000000,42.000000
max,100.000000,3300.000000


In [62]:

# nb of wine rated
nb_rated = df2["points"].count()
nb_rated

120975

In [63]:
# nb of different countries
nb_country = df2["country"].nunique()
nb_region = df2["province"].nunique()
print(f"country:{nb_country}, regions:{nb_region}")

country:42, regions:422


In [185]:
# Sample the database based on the wine production on 2016
# loading the data
prod_file = r"Resources/wine_production_by_country_2016.csv"
df_prod = pd.read_csv(prod_file,
                      encoding="utf-8",
                      header=None,
                     names=["Country","Year","production"])
df_prod = df_prod.drop(["Year"], axis=1)
# calculating the total production this year
total_production = df_prod["production"].sum()
total_production
# Normalize the production
df_prod["production"] = df_prod["production"]/total_production
# Replace United State by US to match the main dataset labels
df_prod['Country'] = df_prod['Country'].str.replace("United States", "US")

# We will select only the 10 bigest countries
df_prod2 = df_prod[0:9]
# China has only one wine reviewed in the dataset so we just remove it
df_prod2 = df_prod2.drop(5) 
df_prod2

,Country,production
0,Italy,0.199843
1,France,0.170789
2,Spain,0.154299
3,US,0.093836
4,Australia,0.051040
6,South Africa,0.041225
7,Chile,0.039654
8,Argentina,0.036906


In [97]:
# Selecting only these countries in the main dataset
df3 = df2[df2["country"].isin(df_prod['Country'][0:9])].reset_index()
df3.count()
df3.head()

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,None,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
4,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,None,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo


In [98]:
df3.describe()

,index,points,price
count,107288.000000,107288.000000,107288.000000
mean,65065.465933,88.365866,36.065003
std,37509.313046,3.064732,41.192884
min,2.000000,80.000000,4.000000
25%,32687.750000,86.000000,18.000000
50%,65153.000000,88.000000,26.000000
75%,97557.250000,91.000000,44.000000
max,129970.000000,100.000000,3300.000000


In [101]:
# Extract the year from the description and add it in a new column
#-----------------------------------------------------------------
# selecting the title column
df4 = df3['title'].iloc[0:1000]
df3["Year"]= ""
df3.head()

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,Year
0,2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,
1,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,
2,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,
3,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,None,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,
4,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,None,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,


In [114]:
# Looking for the year in the title and adding it to the column
ii = 0
for row in df3["title"]:
    # Print some index value to follow the evolution of the loop
    if ii in np.arange(1000,130000, 1000):
        print(ii)
    # Using regex to look for a year compatible set of number in the text
    year = re.findall('.*([1-3][0-9]{3})', row)
    # If a year value is found: adding it to the dataframe
    if year:
        df3["Year"].iat[ii] = year[0]
    ii = ii+1

df3.head()        
        

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000


,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,Year
0,2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
1,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
2,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012
3,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,None,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011
4,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,None,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013


In [119]:
# remove the row that dont have year values
df4 = df3.dropna(subset=['Year']).reset_index()
df4.count()

# Saving the dataset into a new json-file
df4.to_json("Resources_clean/winemag_full.json",orient='records')

In [160]:
df4.groupby("country").count()

,level_0,index,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,Year
country,,,,,,,,,,,,,,,
Argentina,3756,3756,3756,2863,3756,3756,3756,3703,0,3753,3753,3756,3756,3756,3756
Australia,2294,2294,2294,1618,2294,2294,2294,2292,0,2007,2007,2294,2294,2294,2294
Chile,4416,4416,4416,3825,4416,4416,4416,0,0,4305,4305,4416,4415,4416,4416
China,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1
France,17776,17776,17776,11840,17776,17776,17776,17708,0,17525,17525,17776,17776,17776,17776
Italy,16914,16914,16914,11993,16914,16914,16914,16889,0,10121,10121,16914,16914,16914,16914
South Africa,1293,1293,1293,718,1293,1293,1293,0,0,1220,1220,1293,1293,1293,1293
Spain,6573,6573,6573,5257,6573,6573,6573,6565,0,6509,6509,6573,6573,6573,6573
US,54265,54265,54265,36751,54265,54265,54265,53989,50292,37511,34535,54265,54265,54265,54265


In [196]:
# reducing the size of the dataset to match the wine production per country

total_nb_rows = 30000

sampled = []
df_temp = []
# sample_size = []
for country in df_prod2["Country"]:
    # Defining the size of the country sample based on its production
    sample_size = int(np.round(df_prod2.loc[df_prod2["Country"]==country]["production"]*total_nb_rows))
    print(f"Country:{country}, Sample size:{sample_size}")
    sampled.append(df4.loc[df4["country"]==country,:].sample(sample_size,replace=False))

merged_dt = pd.concat(sampled, sort=False)
merged_dt.count()


Country:Italy, Sample size:5995
Country:France, Sample size:5124
Country:Spain, Sample size:4629
Country:US, Sample size:2815
Country:Australia, Sample size:1531
Country:South Africa, Sample size:1237
Country:Chile, Sample size:1190
Country:Argentina, Sample size:1107


level_0                  23628
index                    23628
country                  23628
description              23628
designation              16758
points                   23628
price                    23628
province                 23628
region_1                 21135
region_2                  2609
taster_name              19960
taster_twitter_handle    19809
title                    23628
variety                  23628
winery                   23628
Year                     23628
dtype: int64

In [197]:
merged_dt.to_json("Resources_clean/winemag_23000.json",orient='records')